# Neural Networks Classification

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# fix_yahoo_finance is used to fetch data 
import fix_yahoo_finance as yf
yf.pdr_override()

In [2]:
# input
symbol = 'AMD'
start = '2014-01-01'
end = '2018-08-27'

# Read data 
dataset = yf.download(symbol,start,end)

# View Columns
dataset.head()

[*********************100%***********************]  1 of 1 downloaded


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-01-02,3.85,3.98,3.84,3.95,3.95,20548400
2014-01-03,3.98,4.00,3.88,4.00,4.00,22887200
2014-01-06,4.01,4.18,3.99,4.13,4.13,42398300
2014-01-07,4.19,4.25,4.11,4.18,4.18,42932100
2014-01-08,4.23,4.26,4.14,4.18,4.18,30678700


In [3]:
dataset = dataset.reset_index()
dataset.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-01-02,3.85,3.98,3.84,3.95,3.95,20548400
1,2014-01-03,3.98,4.00,3.88,4.00,4.00,22887200
2,2014-01-06,4.01,4.18,3.99,4.13,4.13,42398300
3,2014-01-07,4.19,4.25,4.11,4.18,4.18,42932100
4,2014-01-08,4.23,4.26,4.14,4.18,4.18,30678700


In [4]:
dataset['Increase_Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,0)
dataset['Buy_Sell'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],1,0)
dataset['Returns'] = dataset['Adj Close'].pct_change()
dataset = dataset.dropna()

Data Exploration

In [5]:
dataset.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns
1,2014-01-03,3.98,4.00,3.88,4.00,4.00,22887200,1,1,1,0.012658
2,2014-01-06,4.01,4.18,3.99,4.13,4.13,42398300,1,1,1,0.032500
3,2014-01-07,4.19,4.25,4.11,4.18,4.18,42932100,0,1,0,0.012107
4,2014-01-08,4.23,4.26,4.14,4.18,4.18,30678700,0,0,0,0.000000
5,2014-01-09,4.20,4.23,4.05,4.09,4.09,30667600,0,0,1,-0.021531


In [6]:
dataset.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns
1167,2018-08-21,19.980000,20.420000,19.860001,20.400000,20.400000,55629000,1,1,1,0.021021
1168,2018-08-22,20.280001,20.920000,20.209999,20.900000,20.900000,62002700,1,1,1,0.024510
1169,2018-08-23,21.190001,22.320000,21.139999,22.290001,22.290001,113444100,1,1,1,0.066507
1170,2018-08-24,22.910000,24.000000,22.670000,23.980000,23.980000,164328200,1,1,1,0.075819
1171,2018-08-27,24.940001,27.299999,24.629999,25.260000,25.260000,325058400,0,0,0,0.053378


In [7]:
dataset.dtypes

Date                 datetime64[ns]
Open                        float64
High                        float64
Low                         float64
Close                       float64
Adj Close                   float64
Volume                        int32
Increase_Decrease             int32
Buy_Sell_on_Open              int32
Buy_Sell                      int32
Returns                     float64
dtype: object

In [8]:
from sklearn.neural_network import MLPClassifier

In [13]:
X = dataset.drop(['Date','Buy_Sell'],axis=1)
y = dataset['Buy_Sell']

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [15]:
# Fit only to the training data
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [16]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [17]:
mlp = MLPClassifier(hidden_layer_sizes=(13,13,13),max_iter=500)

In [18]:
mlp.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(13, 13, 13), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

Predictions and Evaluation

In [19]:
predictions = mlp.predict(X_test)

In [20]:
from sklearn.metrics import classification_report,confusion_matrix


print(confusion_matrix(y_test,predictions))

[[92 46]
 [73 82]]


In [21]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.56      0.67      0.61       138
          1       0.64      0.53      0.58       155

avg / total       0.60      0.59      0.59       293



In [22]:
len(mlp.coefs_)

4

In [23]:
mlp.coefs_

[array([[-0.1881383 ,  0.06936781,  0.25132071,  0.54200624,  0.24556997,
          0.04277295,  0.17382716, -0.46497638, -0.21438311,  0.18281472,
          0.15372843, -0.02028545, -0.52916784],
        [ 0.21992635, -0.00838443,  0.15562097, -0.40814875, -0.0787755 ,
          0.24326174,  0.15224143, -0.34738608, -0.12321794, -0.15675869,
          0.03130395, -0.27681512,  0.22712127],
        [-0.43183095,  0.19240544, -0.25027285, -0.35697896, -0.17613975,
          0.1664433 ,  0.04653116, -0.02330485,  0.54639837,  0.20604981,
         -0.18547052,  0.04349312,  0.32171182],
        [ 0.07242393, -0.36008638, -0.4151522 , -0.15397501,  0.47022404,
         -0.11808914, -0.3989602 ,  0.05111261,  0.01755496, -0.44579156,
          0.16990582,  0.22563828,  0.24117851],
        [-0.18798663,  0.1655963 , -0.19847794, -0.28466721, -0.03540747,
         -0.34743287, -0.23462429,  0.31911658, -0.05346275,  0.48205172,
          0.40541368, -0.46273255,  0.0809665 ],
        [-0.446

In [24]:
len(mlp.coefs_[0])

9

In [25]:
mlp.coefs_[0]

array([[-0.1881383 ,  0.06936781,  0.25132071,  0.54200624,  0.24556997,
         0.04277295,  0.17382716, -0.46497638, -0.21438311,  0.18281472,
         0.15372843, -0.02028545, -0.52916784],
       [ 0.21992635, -0.00838443,  0.15562097, -0.40814875, -0.0787755 ,
         0.24326174,  0.15224143, -0.34738608, -0.12321794, -0.15675869,
         0.03130395, -0.27681512,  0.22712127],
       [-0.43183095,  0.19240544, -0.25027285, -0.35697896, -0.17613975,
         0.1664433 ,  0.04653116, -0.02330485,  0.54639837,  0.20604981,
        -0.18547052,  0.04349312,  0.32171182],
       [ 0.07242393, -0.36008638, -0.4151522 , -0.15397501,  0.47022404,
        -0.11808914, -0.3989602 ,  0.05111261,  0.01755496, -0.44579156,
         0.16990582,  0.22563828,  0.24117851],
       [-0.18798663,  0.1655963 , -0.19847794, -0.28466721, -0.03540747,
        -0.34743287, -0.23462429,  0.31911658, -0.05346275,  0.48205172,
         0.40541368, -0.46273255,  0.0809665 ],
       [-0.44612396, -0.117797

In [26]:
len(mlp.intercepts_[0])

13

In [27]:
mlp.intercepts_[0]

array([-0.48337664,  0.44546505,  0.40579246, -0.13866209, -0.28064258,
        0.30188899, -0.13808039, -0.06443187,  0.09557687, -0.44393233,
        0.13043914, -0.22763605,  0.29033847])

In [30]:
print("weights between input and first hidden layer:")
print(mlp.coefs_[0])
print("\nweights between first hidden and second hidden layer:")
print(mlp.coefs_[1])

weights between input and first hidden layer:
[[-0.1881383   0.06936781  0.25132071  0.54200624  0.24556997  0.04277295
   0.17382716 -0.46497638 -0.21438311  0.18281472  0.15372843 -0.02028545
  -0.52916784]
 [ 0.21992635 -0.00838443  0.15562097 -0.40814875 -0.0787755   0.24326174
   0.15224143 -0.34738608 -0.12321794 -0.15675869  0.03130395 -0.27681512
   0.22712127]
 [-0.43183095  0.19240544 -0.25027285 -0.35697896 -0.17613975  0.1664433
   0.04653116 -0.02330485  0.54639837  0.20604981 -0.18547052  0.04349312
   0.32171182]
 [ 0.07242393 -0.36008638 -0.4151522  -0.15397501  0.47022404 -0.11808914
  -0.3989602   0.05111261  0.01755496 -0.44579156  0.16990582  0.22563828
   0.24117851]
 [-0.18798663  0.1655963  -0.19847794 -0.28466721 -0.03540747 -0.34743287
  -0.23462429  0.31911658 -0.05346275  0.48205172  0.40541368 -0.46273255
   0.0809665 ]
 [-0.44612396 -0.11779751  0.10020353 -0.17426744 -0.13106907  0.51238328
   0.19844734  0.24572122  0.33676495 -0.40597193 -0.59210846 -0.2

In [32]:
print("w0 = ", mlp.coefs_[0][0][0])
print("w1 = ", mlp.coefs_[0][1][0])

w0 =  -0.18813830107912635
w1 =  0.21992634931338156


In [33]:
print("Bias values for first hidden layer:")
print(mlp.intercepts_[0])
print(mlp.intercepts_[1])

Bias values for first hidden layer:
[-0.48337664  0.44546505  0.40579246 -0.13866209 -0.28064258  0.30188899
 -0.13808039 -0.06443187  0.09557687 -0.44393233  0.13043914 -0.22763605
  0.29033847]

Bias values for second hidden layer:
[-0.10386124 -0.36627652  0.11530672 -0.1511756  -0.50311608 -0.10249083
  0.31587765 -0.46692696  0.19410117 -0.41689523  0.52862756 -0.5506964
  0.44603973]
